In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import zipfile
import os

zip_ref = zipfile.ZipFile('/content/sensors_datasets.zip','r') #Opens the zip file
zip_ref.extractall('/tmp') #Extracts the files into the /tmp folder
zip_ref.close()

In [ ]:
salt_dir = "/tmp/sensors_datasets/Salt_treaments_MLdatasets.xlsx"
acetic_acid_dir = "/tmp/sensors_datasets/Acetic_acid_treatment_MLdatasets.xlsx"

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_curve, auc, mean_squared_error, mean_absolute_error
import numpy as np
import tensorflow as tf

In [ ]:
tf.compat.v1.losses.sparse_softmax_cross_entropy

<function tensorflow.python.ops.losses.losses_impl.sparse_softmax_cross_entropy(labels, logits, weights=1.0, scope=None, loss_collection='losses', reduction='weighted_sum_by_nonzero_weights')>

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# import tensorflow_hub as hub
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import cv2 as cv
from torch.utils.data import DataLoader

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, roc_auc_score, mean_squared_error
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import roc_curve

In [ ]:
from torchvision import transforms, models
from torch.utils.data import DataLoader, Dataset
import torch.optim as optim
from torchvision.transforms import ToTensor
import matplotlib.pyplot as plt
from torch.optim.lr_scheduler import OneCycleLR

In [ ]:
import os
import pandas as pd

In [ ]:
pip install openpyxl

In [ ]:
plant_datasets= "/tmp/sensors_datasets"
salt_dir = plant_datasets + "/Salt_treaments_MLdatasets"
acetic_acid_dir = plant_datasets + "/Acetic_acid_treatment_MLdatasets"

In [ ]:
# List all Excel files in the folder
excel_files = [file for file in os.listdir(plant_datasets) if file.endswith('.xlsx')]

In [ ]:
# Initialize a dictionary to store unique plant information
plant_data = {}

for file in excel_files:
    # Construct the full file path
    file_paths = os.path.join(plant_datasets, file)

    # Load the Excel file
    df = pd.read_excel(file_paths, header=None)

    # Initialize an empty DataFrame to store the reshaped data
    reshaped_df = pd.DataFrame()

    # Iterate over the rows to find plant names, treatments, and sensor numbers
    for index, row in df.iterrows():
        # Check if the cell contains a plant name, treatment, and sensor number
        if isinstance(row[0], str) and 'CHT' in row[0]:
            plant_info = row[0].split(' ')
            plant_name = plant_info[0]
            treatment = plant_info[1]
            sensor_number = plant_info[2]

            # Create a unique key for each plant type
            plant_key = f"{plant_name}_{treatment}_{sensor_number}"

            # Add the extracted data to the plant_data dictionary
            if plant_key not in plant_data:
                plant_data[plant_key] = []

            # Initialize a list to store data for the current plant key
            plant_data_list = []

            # Extract data for each hour of the day
            for hour_index in range(1, 25):  # Assuming 24 hours of data
                hour_data = df.iloc[index + hour_index].tolist()
                hour = hour_data[0]  # Assuming the first cell is the hour
                parameters = hour_data[1:]  # Assuming the rest of the cells are the parameters
                plant_data[plant_key].append({'hour': hour, 'parameters': parameters})

    # Print the plant data
    for plant_key, data in plant_data.items():
        print(f"Plant: {plant_key}")
        for hour_data in data:
            print(f"Hour: {hour_data['hour']}, Parameters: {hour_data['parameters']}")
        print("\n")

In [ ]:
# Function to calculate variation and categorize it for each set of four parameters
def categorize_variation(hour_data):
    variations = {}
    categories = {}

    # Iterate over each hour in the data
    for hour_data in data:
        hour = hour_data['hour']
        parameters = hour_data['parameters']
        hour_variations = []
        hour_categories = []

        for i in range(0, len(parameters), 4):
            param_set = [parameters[i + 1], parameters[i + 3]]

            # Check if all values in the set are numeric
            if all(isinstance(x, (int, float)) for x in param_set):
                variation = pd.Series(param_set).std() / pd.Series(param_set).mean()
                hour_variations.append(variation)

                # Categorize the variation
                if variation <= 1.13:
                    category = 1  # good
                elif variation <= 1.17:
                    category = 2  # medium
                else:
                    category = 3  # bad
                hour_categories.append(category)
            else:
                # Skip the calculation for non-numeric parameter sets
                hour_variations.append(None)
                hour_categories.append(None)

        variations[hour] = hour_variations
        categories[hour] = hour_categories

    return variations, categories

for plant_key, data in plant_data.items():
    print(f"Plant: {plant_key}")
    variations, categories = categorize_variation(data)
    for hour, hour_variations in variations.items():
        print(f"Hour: {hour}, Variations: {hour_variations}, Categories: {categories[hour]}")
    print("\n")

In [ ]:
# Function to normalize data
def normalize_data(df, scaler):
    df[['temperature', 'moisture', 'soil_fertility', 'light_intensity']] = scaler.transform(df[['temperature', 'moisture', 'soil_fertility', 'light_intensity']])
    return df

In [ ]:
# Function to create sequences for LSTM
def create_sequences(df, n_steps=1):
    X, y = [], []
    for i in range(len(df) - n_steps):
        X.append(df[['temperature', 'moisture', 'soil_fertility', 'light_intensity']].iloc[i:(i + n_steps)].values)
        y.append(df['category'].iloc[i + n_steps])
    return np.array(X), np.array(y)

In [ ]:
# Load and preprocess data
df_salt = categorize_variation(salt_dir)
df_acetic_acid = categorize_variation(acetic_acid_dir)

In [ ]:
train_df_salt, temp_df_salt = train_test_split(df_salt, test_size=0.3, random_state=42)

# Check if temp_df_salt has enough samples for further splitting
if len(temp_df_salt) > 1:
    val_df_salt, test_df_salt = train_test_split(temp_df_salt, test_size=0.33, random_state=42)
else:
    # Handle the case where there are not enough samples for further splitting
    print("Not enough samples in temp_df_salt for further splitting. Consider adjusting test_size or using a different approach.")

Not enough samples in temp_df_salt for further splitting. Consider adjusting test_size or using a different approach.


In [ ]:
# Adjust the test_size to ensure temp_df_salt has at least 3 samples
train_df_salt, temp_df_salt = train_test_split(df_salt, test_size=0.4, random_state=42)

# Check the number of samples in temp_df_salt
if len(temp_df_salt) >= 3:
    val_df_salt, test_df_salt = train_test_split(temp_df_salt, test_size=0.5, random_state=42)
else:
    print("Not enough samples in temp_df_salt for further splitting.")

Not enough samples in temp_df_salt for further splitting.


In [ ]:
print("Size of df_salt:", len(df_salt))

Size of df_salt: 2


In [ ]:
# Split the data
train_df_salt, temp_df_salt = train_test_split(df_salt, test_size=0.3, random_state=42)
val_df_salt, test_df_salt = train_test_split(temp_df_salt, test_size=1/3, random_state=42)

train_df_acetic_acid, temp_df_acetic_acid = train_test_split(df_acetic_acid, test_size=0.3, random_state=42)
val_df_acetic_acid, test_df_acetic_acid = train_test_split(temp_df_acetic_acid, test_size=1/3, random_state=42)

ValueError: With n_samples=1, test_size=0.3333333333333333 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.